<a href="https://colab.research.google.com/github/raketic-ognjen/introtoportfolioopt2/blob/main/Introtoportfolioopt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='tomato' style="font-size:40px"><center><b>Introduction to Portfolio Optimization</b></center></font><font color='DarkOrange' style="font-size:30px"><center><b>- Part 2 -</b></center></font>

## <font color='orange' style="font-size:25px"><b>Installing and updating relevant packages</b></font>

As usual, I need to update several packages which older version already exist in Google Colab:

In [ ]:
%pip install plotly

Before I start today's coding practice, it is necessary to import following packages:
* <font color='mediumseagreen'><b>Pandas</b></font>
* <font color='mediumseagreen'><b>NumPy</b></font>
* <font color='mediumseagreen'><b>Plotly</b></font>
* <font color='mediumseagreen'><b>SciPy - Stats</b></font>
* <font color='mediumseagreen'><b>SciPy - Optimize</b></font>
* Our small package for investment lectures - <font color='DarkTurquoise'><b>quant formulas</b></font>. Before we import it, we need to upload it, of course:

In [ ]:
from google.colab import files
uploaded0=files.upload()

Saving kvantitativne_formule.py to kvantitativne_formule.py


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import scipy.stats as st
import scipy.optimize as sco
import kvantitativne_formule as qt

Also, before I start, I need to prepare data in the same way as I did in all my repositories. Thus, let me first import file "optimization.xlsx":

In [ ]:
from google.colab import files
uploaded1= files.upload()

Saving optimization.xlsx to optimization.xlsx


In [ ]:
prices=pd.read_excel('optimization.xlsx',sheet_name='Sheet1',index_col="DATE")
prices

,AMZN,MSFT,IBM,AAPL,^GSPC,rf
DATE,,,,,,
2013-1-2,257.309998,27.620001,196.350006,78.432899,1462.420044,0.000007
2013-1-3,258.480011,27.250000,195.270004,77.442299,1459.369995,0.000007
2013-1-4,259.149994,26.740000,193.990005,75.285698,1466.469971,0.000007
2013-1-7,268.459015,26.690001,193.139999,74.842903,1461.890015,0.000006
2013-1-8,266.380005,26.549999,192.869995,75.044296,1457.150024,0.000006
...,...,...,...,...,...,...
2014-12-24,303.029999,48.139999,161.820007,112.010002,2081.879883,0.000005
2014-12-26,309.089996,47.880001,162.339996,113.989998,2088.770020,0.000005
2014-12-29,312.040009,47.450001,160.509995,113.910004,2090.570068,0.000005


In [ ]:
modellingP=prices[:300] #the first 300 obervations
testingP=prices[300:] #rest

In [ ]:
retM=modellingP.iloc[:,:4].pct_change()
retM.dropna(inplace=True)
retM.head()

,AMZN,MSFT,IBM,AAPL
DATE,,,,
2013-1-3,0.004547,-0.013396,-0.005500,-0.012630
2013-1-4,0.002592,-0.018716,-0.006555,-0.027848
2013-1-7,0.035921,-0.001870,-0.004382,-0.005882
2013-1-8,-0.007744,-0.005245,-0.001398,0.002691
2013-1-9,-0.000113,0.005650,-0.002852,-0.015629


## <font color='Orange' style="font-size:25px"><b>Efficient Frontier</b></font>

We start, for simplicity, with an example of 2 stocks. We assume that no short selling is allowed, i.e. that weights are between 0 and 1. **You can read Bodie, Kane, Markus (pp 197-205) for some additional insights into the problem with two assets.**

### <font color='MediumVioletRed' style="font-size:20px"><b>Initial discussion - portfolio possibilities curve</b></font>

Assume that you have only two stocks in your portfolio. Variances of their returns are given as: $\mathbb{V}\left(r_{1}\right)=\sigma_1^2$ and $\mathbb{V}\left(r_{2}\right)=\sigma_2^2$, while covariance of their returns is $\sigma_{1,2}$. In this case, portfolio's variance would be (recall that $\alpha_1 + \alpha_2 = 1$):

$$\mathbb{V}(r_{p})= \alpha_1^2\sigma_1^2+\alpha_2^2\sigma_2^2+2\,\alpha_1\,\alpha_2\,\sigma_{1,2}$$

Note that covariance (just like the expected returns or standard deviations) depend on the time units that we use. If we want to have a measure of linear statistical dependence which does not depend on units we use correlation coefficients. Correlations are always between -1 (perfect negative correlation) to +1 (perfect positive correlation).

 Correlation coefficient $\rho$ is defined as:

$$\rho=\frac{\sigma_{1,2}}{\sigma_1\,\sigma_2}\,\,\,\,\,\,\,\,\,\, \Longleftrightarrow\,\,\,\,\,\,\,\,\,\, \sigma_{1,2}=\,\rho\, \sigma_1\,\sigma_2$$

Thus, the formula for portfolio's variance of a two-asset portfolio can be rewritten as (recall that the sum of the 2 weightes needs to be equal to 1):

$$\mathbb{V}(r_{p})= \alpha_1^2\sigma_1^2+\alpha_2^2\sigma_2^2+2\,\alpha_1\,\alpha_2\,\rho\sigma_1\,\sigma_2 = \alpha_1^2\sigma_1^2+(1 -\alpha_1)^2\sigma_2^2+2\,\alpha_1\,(1-\alpha_1)\,\rho\sigma_1\,\sigma_2$$

Our further conclusions will depend on $\rho$. The expected return on portfolio is

$$\mathbb{E}(r_{p}) = \alpha_1 \mathbb{E}(r_{1}) + \alpha_2 \mathbb{E}(r_{2}) = \alpha_1 \mu_1 + (1-\alpha_1) \mu_2$$  

Suppose we consider all possible portfolios corresponding to $\alpha_1$ taking values from 0 to 1 and that for each of these portfolios we plot corresponding values of standard deviation (on x axis) and the expected return (on y axis). This is what we construct next.

#### <font color='MediumPurple' style="font-size:16px"><b>Perfect negative correlation</b></font>

Assume for a moment that the returns on two assets in our portfolio are perfectly negatively correlated (i.e. $\rho=-1$). In that case, the expression for portfolio variance becomes:

$$\mathbb{V}(r_{p})= \alpha_1^2\sigma_1^2+\alpha_2^2\sigma_2^2-2\,\alpha_1\,\alpha_2\,\sigma_1\,\sigma_2=(\alpha_1\,\sigma_1-\alpha_2\,\sigma_2)^2$$

$$\sigma_p=\sqrt{\,\mathbb{V}(r_{p})\,}=|\alpha_1\,\sigma_1-\alpha_2\,\sigma_2|$$

The following graphical display compares risk and reward for all possible portfolios. This is why it is called **portfolio possibilities curve/line**. Let us plot assuming that $\sigma_1=0.12$, $\sigma_2=0.2$, $\mu_1=0.08$ and $\mu_2=0.13$.

In [ ]:
sigma_1 = 0.12
sigma_2 = 0.2
mu_1 = 0.08
mu_2=0.13

In [ ]:
w= np.linspace(0,1,100)
pvar = np.abs(w*sigma_1 - (1-w)*sigma_2)
pe = w*mu_1+(1-w)*mu_2
fig=go.Figure()
fig.add_trace(go.Scatter(x=pvar,y=pe))
fig.update_layout(xaxis=dict(title_text='\$\sigma$', range=[-0.01,0.25],zerolinecolor="Black"),yaxis=dict(title_text='\$\mu$'),
                  title=dict(text="performance of possible portfolios for rho-1",x=0.5,y=0.87, font=dict(size=25,color='Green')))

fig.show()

Suppose that the goal is to minimize portfolio’s volatility i.e. risk. As you can see from the graph above, if two risky stocks are perfectly negatively correlated we can construct a completely riskless portfolio using the two risky assets!

Knowing that volatilities $\sigma_1$ and $\sigma_2$ are both positive numbers, the condition that $\sigma_p=0$ implies that:

$$\alpha_1\,\sigma_1-(1-\alpha_1)\,\sigma_2=0$$

Solving this equation with respect to $\alpha_1$:

$$\alpha_1=\frac{\sigma_2}{\sigma_1+\sigma_2}$$

In [ ]:
alpha_1 = sigma_2/(sigma_1+sigma_2) # Fraction of money invested at asset 1 for which portfolio risk is 0
alpha_1

0.625

and since $\alpha_2=(1-\alpha_1)$, the optimal $\alpha_2$ is:
$$\alpha_2=\frac{\sigma_1}{\sigma_1+\sigma_2}$$

If we invest into the stocks using the above proportions we obtain minimum variance portfolio or MVP in case when the opportunity set consists of two perfectly negatively correlated stocks and the corresponding (ex-ante) portfolio risk is 0!

#### <font color='MediumPurple' style="font-size:16px"><b>Perfect positive correlation</b></font>

Now assume that the two stocks are perfectly  positively correlated (i.e. $\rho=1$). Portfolio's variance in this case is:

$$\mathbb{V}(r_{p})= \alpha_1^2\sigma_1^2+\alpha_2^2\sigma_2^2+2\,\alpha_1\,\alpha_2\,\sigma_1\,\sigma_2=(\alpha_1\,\sigma_1+\alpha_2\,\sigma_2)^2$$

So, volatility of this portfolio is:

$$\sigma_p=\sqrt{\,\mathbb{V}(r_{p})\,}=\alpha_1\,\sigma_1+(1-\alpha_1)\,\sigma_2$$

Note that here we don't have the absolute value since the expression above is always positive (as long as short-selling is not allowed):

In [ ]:
w = np.linspace(0,1,100)
pvar = w*0.12+(1-w)*0.2
pe=w*0.08+(1-w)*0.13

fig=go.Figure()
fig.add_trace(go.Scatter(x=pvar,y=pe,line_color = 'Crimson'))
fig.update_layout(xaxis=dict(title_text='\$\sigma$', range=[-0.01,0.25], zerolinecolor='Black'),yaxis=dict(title_text='$\mu$'),
                  title=dict(text="Performance of possible portfolio when r=1", x=0.5,y=0.87,font=dict(size=25,color='Navy')))

fig.show()

In this case the relation is the straight line which does not touch the y axis. This implies that we cannot create a portfolio with zero volatility as in the previous case **(can you think of a caveat when this is not true?)**.

So, what is the lowest level of risk that we can get in this case when all our stocks are perfectly positively correlated? Since:

$$\sigma_p=\alpha_1\,\sigma_1+(1-\alpha_1)\,\sigma_2$$

Note that for $\alpha_1=1$, $\sigma_p=\sigma_1$, while for $\alpha_2=1$, $\sigma_p=\sigma_2$. Since portfolio volatility is a straight line between $\sigma_1$ and $\sigma_2$, all points in between (i.e. volatilities of all other portfolios) are their linear combination. Thus, the minimum of this function is: $\min(\sigma_1,\,\sigma_2)$.

In case of a perfect positive correlation, there is no benefits of diversification. To minimize volatility one would need to invest only in the stock with the smallest volatility. Any combination of two stocks would result in a portfolio with higher volatility.

When correlation is -1 the benefits of diversification are enormous (we can get risk of the entire risk).

#### <font color='MediumPurple' style="font-size:16px"><b>Imperfect correlation - the real life case</b></font>

This example deals with the case when correlation coefficient $\rho$ is between -1 and 1. Let us plot the portfolio possibilities curve for all possible $\rho$'s.

In [ ]:
w=np.linspace(0,1,100)
r=np.linspace(-1,1,201)
pvar1=w*0.12+(1-w)*0.2
pvar2=np.abs(w*0.12-(1-w)*0.2)
pe=w*0.08+(1-w)*0.13
fig=go.Figure()
fig.add_trace(go.Scatter(x=pvar1,y=pe,line_color='DeepSkyBlue'))
fig.add_trace(go.Scatter(x=pvar2,y=pe,line_color='Crimson'))
for i in range(201):
    pvar=np.sqrt(w**2*0.12**2+(1-w)**2*0.2**2+2*r[i]*0.2*0.12*w*(1-w))
    fig.add_trace(go.Scatter(x=pvar,y=pe,line_color='Gold',visible=False))
fig.data[2].visible=True

steps = [dict(method="update",label=round(r[i-2],2),args=[{"visible": [True,True]+[t==i for t in range(2,len(fig.data))]},
                                        {"title": "Correlation is: "+str(round(r[i-2],2))}]) for i in range(2,len(fig.data))]
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),yaxis=dict(title_text='$\mu$'),
                  title=dict(text="Performance of possible portfolios",x=0.5,y=0.87,font=dict(size=25,color='Navy')),
                  showlegend=False,sliders=[dict(active=0,steps=steps)])

fig.show()

As you can note from the graph, this line, which represent relationship between risk and reward for all possible portfolios, is always between two the extreme cases - perfect positive and perfect negative correlation.

### <font color='MediumVioletRed' style="font-size:20px"><b>More than two stocks - from the portfolio possibilities curve to the efficient frontier</b></font>

In case when we have more than two stocks, **portfolio possibilities set** forms a 2D area in $\sigma,\mu$ plane. These are portfolios that we can construct given our investment opportunity set. But, some of these portfolios are unambiguously better than others in terms of the risk-return tradeoff.

Our goal is to find all portfolios that are the **efficient** ones in terms of risk-return tradeoff. For an efficient portfolio, we cannot find another possible portfolio that has the same risk and, at the same time, higher expected return than that portfolio. Alternatively looked, for a given level of the the expected return, we cannot find a portfolio with the smaller risk.

These portfolios are obtained in the process of mean-variance optimization. In practice, we take target expected return $\mu$ as a parameter that we vary and find portfolios that have expected return $\mu$ realized with smallest volatility (or variance).

We have seen these kinds of calculations for a given target return $\mu$ in the previous lecture. Now we do it, in principle, for all possible values of target return $\mu$.

To present the efficient portfolios graphically in the risk-expected return framework, we need to calculate optimal portfolio weights for a variety of target expected returns (for example from 1% to 40%). After that, we need to plot relationship between their risk and reward graphically in the same way as we did before.


In our first example, for pedagogical reasons, let's consider the first three stocks in our opportunity set.

We create a loop which determines optimal portfolios for different target expected returns, and stores their risk (volatility) and reward (expected returns) in two lists or <font color='DodgerBlue'><b>np.arrays</b></font>.

In [ ]:
retM3=retM.iloc[:,:3] #Selecting the first three stocks
mi = np.arange(0.01,0.41,0.01) # Target expected returns
sigma = np.array([qt.targetP(retM3,m)['vol'] for m in mi]) # Find minimum volatility for a given expected return mi

In [ ]:
fig= go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

All portfolios on this curve are the results of optimization! All other possible (i.e. achievable) portfolios which do not belong to this curve are inefficient by default, i.e. they are not the result of the optimization (the area painted in Pink in the graph below).

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3),fill="toself",fillcolor='Pink'))
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3),fill="tozerox",fillcolor='LightGreen'))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

This curve together with the pink area forms the portfolio possibilities set. Green area on the graph cannot be achieved with the opportunity set that we have. I.e. this is outside of the portfolio possibilities set (**but it would be great to be there, if we only could!**)

Since this curve separates possible from impossible to achieve portfolios (it is in a way a frontier between the two sets) and since on this curve we have efficient portfolios, it is called **efficient frontier**! But, there is an important point to clarify here.

Can anybody guess where MVP is on this graph? If you remember the definition of MVP, it is the portfolio with the global minimum of risk. In other words, you cannot get smaller risk than that. So, it is obvious that MVP is following point:

In [ ]:
MVP3=qt.mvp(retM3)

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
fig.add_trace(go.Scatter(x=[MVP3['vol']],y=[MVP3['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

An investor who, *ceteris paribus*, prefers higher returns to lower returns and smaller risks to higher risks would always prefer to be on this curve rather than in any other part of portfolio possibilities set.

The reason for this is simple - for the same level of risk investor can get higher reward if he/she moves on the efficient frontier. But, are all points on the efficient frontier really efficient?

Take a look at a graph below:

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3),fill="toself",fillcolor='Pink'))
fig.add_trace(go.Scatter(x=[0.159]*2,y=[0.045,0.2],line=dict(color='Black',width=3)))
fig.add_trace(go.Scatter(x=[0,0.159],y=[0.045,0.045],line=dict(color='Black',width=3,dash='dash')))
fig.add_trace(go.Scatter(x=[0,0.159],y=[0.2,0.2],line=dict(color='Black',width=3,dash='dash')))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
            yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
            title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')),
            annotations=[dict(text="A=(15.9%,20%)",x=0.159,axref='x',ayref='y',y=0.2,ax=0.11,ay=0.3,font_size=20,
                          arrowhead=2,arrowcolor='Green',arrowwidth=3),
                        dict(text="B=(15.9%,4.5%)",x=0.159,axref='x',ayref='y',y=0.045,ax=0.11,ay=0.1,font_size=20,
                          arrowhead=2,arrowcolor='Red',arrowwidth=3)])
fig.show()

For the same level of risk (i.e. volatility of 15.9%) we can get higher reward if we invest in portfolio $A$ (expected return is 20%) instead of portfolio $B$ (which expected return is 4.5%)! Thus, portfolio $A$ is strictly prefered to portfolio $B$.

Portfolio $A$ is efficient while $B$ is not. So, not the whole curve obtained via optimization is really efficient.

Only the part of the optimization curve at or above the MVP is really efficient, while the part below the MVP (as well as the rest of portfolio possibilities set) is inefficient (since for the same level of risk you can find another portfolio which offers higher reward).

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',dash='dash'),fill="toself",fillcolor='Pink'))
fig.add_trace(go.Scatter(x=sigma[mi>MVP3['er']],y=mi[mi>MVP3['er']],line=dict(color='Blue',width=3)))
fig.add_trace(go.Scatter(x=[MVP3['vol']],y=[MVP3['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="True efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

Before we continue, we define the function which will either prepare data for plotting the efficient frontier or plot the efficient frontier in different ways (only the curve, the curve and MVP or the curve, MVP and portfolio possibilities set.

We will test this function in the following subsection. qt.EF

### <font color='MediumVioletRed' style="font-size:20px"><b>Adding stocks into the portfolio - diversification effect</b></font>

If we use only the first three stocks in our portfolio optimization we cannot reach some portfolio performances (they were presented with green on one of our previous graphs). But, is there any way in which we can still reach them in a mean-variance optimal way?

One possible approach is to add additional assets into  your portfolio or, in other words, to expand the investment opportunity set. By doing so, you will diversify your portfolio and reduce the risk of your optimal investments for a given level of expected returns. Below we use all 4 stocks.

Adding one more risky asset we can achieve the same level of expected returns with lower risk! Graphically, this means that your efficient frontier is moved to the left (closer to y axis, i.e. closer to zero volatility or closer to certainty).

Let us use our function <font color='Blue'><b>EF</b></font> to plot the efficient frontier in case when we have all four stocks in our portfolio:

In [ ]:
qt.EF(retM3,plot='full')#Efficient frontier with all 4 stocks

Before we move on let me just show you that function can plot efficient frontier if you insert assumptions about expected returns and covariance matrix instead of data (here I assume the same sample based measures of expected returns and covariance matrix that function uses by default, but here we can put some other estimates instead):

In [ ]:
qt.EF(plot='full',er_assumed=qt.expRet(retM),cov_assumed=retM.cov()*252)

You may note that efficient frontier moved closer to the y axis as we expected. Just to be sure, let us plot this new efficient frontier (with 4 stocks) alongside with the old one (with the first 3 stocks only).

First we need data for plotting the new efficient frontier. We can again call our function <font color='Blue'><b>EF</b></font>. Since output is dictionary, we can use method <font color='DeepPink'><b>values</b></font> to unpack output into two variables as in code below:

In [ ]:
sigma4,mi4=qt.EF(retM).values()

Now we have everything that we need to plot this two efficient frontiers side by side (data for efficient frontier with 3 stocks is given in previous subsection):

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3),name='Old (3 stocks)'))
fig.add_trace(go.Scatter(x=sigma4,y=mi4,line=dict(color='Crimson',width=3),name='New (4 stocks)'))
fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,0.25],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

#### <font color='Blue' style="font-size:25px"><b>Another way to plot this graph - Read later</b></font>

Before we move on, let me quickly show you alternative fast way to plot this graph. First, let me remind you that each plotted figure in <font color='mediumseagreen'><b>Plotly</b></font> consists of **data** and **layout**. Thus, when our function <font color='Blue'><b>EF</b></font> plots required figure, we can use attribute <font color='FireBrick'><b>data</b></font> to extract all plotted traces (i.e. tuple in which each element is one trace). In this case, since we have only one trace on the graph this is tuple with one element, let us extract it with slicing:

In [ ]:
trace_4stocks=qt.EF(retM,plot='curve').data[0]
trace_4stocks

Scatter({
    'line': {'color': 'Blue', 'width': 3},
    'x': array([0.15163788, 0.14903257, 0.14666594, 0.14455302, 0.14270507, 0.14113252,
                0.13984466, 0.1388494 , 0.13815308, 0.13776024, 0.13767349, 0.13789341,
                0.13841856, 0.13924551, 0.14036894, 0.14178182, 0.14347554, 0.14544042,
                0.14766542, 0.15013883, 0.15284845, 0.15578197, 0.15892705, 0.16227152,
                0.16580339, 0.169511  , 0.17338309, 0.17740889, 0.18157817, 0.18588127,
                0.19030911, 0.19485318, 0.19950553, 0.20425877, 0.209106  , 0.21404085,
                0.21905739, 0.22415014, 0.22931401, 0.2345443 ]),
    'y': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                0.37, 0.38, 0.39, 0.4 ])
})

Now we can edit this trace (class) in any way that we like, for example we can change some existing values (for example color of the line) or add new ones (name of the curve for example):

In [ ]:
trace_4stocks.line.color='Crimson'
trace_4stocks['name']='New (4 stocks)'

Finally, we can use our function to plot efficient frontier with 3 stocks, and then to add previously edited trace on created figure. This can be faster than to write entire plotting code from the scratch, thus, as you can see, our function is very user-friendly and helpful:

In [ ]:
fig_new=qt.EF(retM3,plot='curve')
fig_new.add_trace(trace_4stocks)
fig_new.data[0]['name']='Old (3 stocks)'
fig_new.show()

### <font color='MediumVioletRed' style="font-size:20px"><b>Domination of covariance in a well-diversified portofolios</b></font>

You can read **Bodie, Kane, Markus, pp. 197-198, 217-219** to understand better the concepts of diversification. Also discussed there is how many assets you need in order to achieve practical level of diversification.

Now let us continue with our analysis. As you can see, the power of diversification lays in the fact that with it we can unlock part of the previously unachievable set. However, we cannot eliminate the whole risk only by diversification. Diversification has its limits. Let me remind you of expression for portfolio's variance:

$$\mathbb{V}(r_{p})=\sum_{j=1}^n \alpha_j^2\,\sigma_j^2+2\,\sum_{1\leq j < i \leq n}^n \alpha_j\,\alpha_i\,\sigma_{j,i}$$

Suppose, for simplicity, that our portfolio is equally weighted, i.e. that each of the $n$ stocks have weights $\alpha_j=\frac{1}{n}$. Portfolio variance in that case reads:

$$\mathbb{V}(r_{p})=\frac{\sum_{j=1}^n \sigma_j^2}{n^2}+\frac{2\,\sum_{1\leq j < i \leq n}^n \sigma_{j,i}}{n^2}$$

We can transform these sums into average values (i.e. average variance and average covariance) by dividing them with the number of elements in them.

In the first sum we have $n$ elements, while in the second sum we have $\frac{n\,(n-1)}{2}$ elements. Bearing this in mind we can write something like this:

\begin{align*}
\mathbb{V}(r_{p}) &=\frac{\frac{n}{n}\sum_{j=1}^n \sigma_j^2}{n^2}+\frac{\frac{\frac{n\,(n-1)}{2}}{\frac{n\,(n-1)}{2}}2\,\sum_{1\leq j \leq i \leq n}^n \sigma_{j,i}}{n^2}\\
&=\frac{\frac{1}{n}\sum_{j=1}^n \sigma_j^2}{n}+\frac{n\,(n-1)\,\left(\frac{1}{\frac{n\,(n-1)}{2}}\,\sum_{1\leq j \leq i \leq n}^n \sigma_{j,i}\right)}{n^2}
\end{align*}

We will label average variance with $\overline{\sigma^2_j}$ and average covariance with $\overline{\sigma}_{j,\,i}$. Thus, expression for portfolio's variance becomes:

\begin{align*}
\mathbb{V}(r_{p,\,t})&=\frac{\overline{\sigma_j^2}}{n}+\frac{n\,(n-1)\,\overline{\sigma}_{j,\,i}}{n^2}\\
&=\frac{\overline{\sigma_j^2}}{n}+\left(1-\frac{1}{n}\right)\,\overline{\sigma}_{j,\,i}
\end{align*}

Assuming that both average values are finite for increasingly large $n$ (a reasonable assumption), what is the limit of this expression when number of stocks of your portfolio goes to infinity?

$$\lim_{n\to\infty}\mathbb{V}(r_{p})=\overline{\sigma}_{j,\,i}$$

Thus, with diversification you can eliminate only part of the risk - risk due to volatility of each individual stock. This is *company-specific* or *idiosyncratic* risk. However, there is part of risk which is nondiversifiable - this is risk due to covariance (i.e. co-movement) of stocks.

So, diversification reduces risk, but with an increase in number the stocks in your portfolio, this reduction of risk is getting smaller and smaller, but it never reaches zero.

Generically speaking, as long as we are dealing with risky assets there is part of the risk which cannot be reduced by adding more and more risky assets into our portfolio. However, full diversification is possible if we add risk-less asset into our portfolio.

## <font color='Orange' style="font-size:25px"><b>Adding risk-free asset</b></font>

In **Bodie, Kane, Markus, pp 206-211** you can read about a example with 2 risky assets and a riskless asset. Much of the ideas expressed here can be understood in this simple example. The key is that with the addition of a riskless asset, the efficient frontier becomes effectively a straight line.

All previously considered assets were risky assets. Assume now that some risk-free asset exists in the economy (assets which offers some returns with virtual certainty). They have variance close to zero and are, in theory, uncorrelated with the rest of the asset returns.

As an approximation of such asset we can take US Treausuries. Their returns are given in column $rf$ (short from risk-free) in our initial modeling data set.

In [ ]:
modellingP.rf.head()

DATE
2013-1-2    0.000007
2013-1-3    0.000007
2013-1-4    0.000007
2013-1-7    0.000006
2013-1-8    0.000006
Name: rf, dtype: float64

Let us join them with the data set of returns on previous four stocks (we are going to use method <font color='DeepPink'><b>join</b></font> and technique called *left join* which we explained in lecture about <font color='mediumseagreen'><b>Pandas</b></font>):

In [ ]:
retMrf=retM.join(modellingP.rf)
retMrf.head()

,AMZN,MSFT,IBM,AAPL,rf
DATE,,,,,
2013-1-3,0.004547,-0.013396,-0.005500,-0.012630,0.000007
2013-1-4,0.002592,-0.018716,-0.006555,-0.027848,0.000007
2013-1-7,0.035921,-0.001870,-0.004382,-0.005882,0.000006
2013-1-8,-0.007744,-0.005245,-0.001398,0.002691,0.000006
2013-1-9,-0.000113,0.005650,-0.002852,-0.015629,0.000006


In theory, risk-free asset should:

1. have zero volatility (since it is risk-free)
2. be uncorrelated with other assets (since they are risky, while this asset is risk-free)

As a drawback, risk-free asset is likely to offer small expected returns since you aren't taking any risk reward should be small. While this is true now, when FED was fighting inflation prime rate in the US was as higher as 18 percent per annum!

Let us check this properties one by one. First let us calculate volatility of all assets in our portfolio including the risk-free asset:

In [ ]:
print("\33[1mVolatility:\33[0m\n", qt.annualize_vol(retMrf))

Volatility:
 AMZN    0.285500
MSFT    0.242170
IBM     0.180507
AAPL    0.279610
rf      0.000019
dtype: float64


As you can see volatility of the risk-free asset is insignificant with respect to that of risky assets. Thus, the first property seems to be satisfied! Let's move on and find the expected retunes on all assets in our portfolio:

In [ ]:
print("\33[1mExpected Returns:\33[0m\n", qt.expRet(retMrf))

Expected Returns:
 AMZN    0.348624
MSFT    0.304543
IBM    -0.028180
AAPL    0.020383
rf      0.001019
dtype: float64


Expected return on risk-free asset is pretty small - it is around 0.1%! Finally, let us calculate the correlation matrix between all assets in our portfolio:

In [ ]:
retMrf.corr()

,AMZN,MSFT,IBM,AAPL,rf
AMZN,1.000000,0.174879,0.199520,-0.013817,-0.014528
MSFT,0.174879,1.000000,0.200167,0.064488,0.021240
IBM,0.199520,0.200167,1.000000,0.119305,-0.019624
AAPL,-0.013817,0.064488,0.119305,1.000000,-0.069519
rf,-0.014528,0.021240,-0.019624,-0.069519,1.000000


#### <font color='Blue' style="font-size:25px"><b>Testing for zero correlation - read on your own</b></font>

Although correlation isn't exactly zero it is pretty small and statistically insignificant. We will show that with Spearman's correlation test. This test is a nonparametric alternative for Pearson's correlation test which requires normality of data in order for it to be accurate (which, of course, clearly isn't satisfied in finance). Hypothesis in every correlation test are:

$$H_0: \rho = 0\,\,\text{(variables are uncorrelated)}$$
$$H_1: \rho > 0\,\,\,\,\,\,\text{(variables are correlated)}$$

In the code below we use function <font color='DodgerBlue'><b>st.spearmanr</b></font> from <font color='mediumseagreen'><b>SciPy - Stats</b></font> to calculate p-value of Spearman's correlation test:

In [ ]:
[st.spearmanr(retMrf.rf,retMrf[i])[1] for i in retM.columns]

[0.9275978308149423,
 0.9790594328321616,
 0.7311329189539367,
 0.36547457586717835]

Since all p-values are greater than 5%, we cannot reject the null hypothesis, thus we can say that the risk-free asset is statistically uncorrelated with the other assets from our small portfolio. So, we can conclude that this approximation of risk-free asset is pretty decent.

### <font color='MediumVioletRed' style="font-size:20px"><b>Optimization with the added risk-free asset</b></font>

Let us use our function <font color='Blue'><b>EF</b></font> to prepare values for plotting the efficient frontier in the case when:
* we have risk-free asset in our portfolio
* we don't have risk-free asset in our portfolio

In [ ]:
sigmaRF,miRF = qt.EF(retMrf,Range=[0.01,0.7]).values() # With the riskfree rate
sigma4,mi4 = qt.EF(retM,Range=[0.01,0.7]).values() #without riskfree rate

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigmaRF,y=miRF,line=dict(color='Blue',width=3),name='4 stocks and rf'))
fig.add_trace(go.Scatter(x=sigma4,y=mi4,line=dict(color='Crimson',width=3),name='4 stocks '))
fig.update_layout(xaxis=dict(title_text='\$\sigma_p$',range=[-0.01,0.4],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu_p$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

By adding the risk-free asset, the efficient frontier becomes a straight line. Furthermore, adding a riskless asset dramatically improves the tradeoff between risk and return. The blue line (new efficient frontier) clearly dominates the old efficient frontier in all points but one.

This point is the **point of tangency** of the old efficient frontier (consisting just of risky assets) and the new one (with an added risk-free asset).

What is the MVP portfolio like when we add the risk-free asset? The answer is simple. MVP is the portfolio in which we put all our money into the risk-free asset (since all other assets are risky, and thus investing into them would increase the overall level of risk in our portfolio). Indeed:

In [ ]:
qt.mvp(retMrf)

{'w': array([-3.84240021e-04,  3.25146722e-04,  1.38826191e-04,  7.60808471e-05,
         9.99844186e-01]),
 'er': 0.000981274827293314,
 'vol': 0.0001296080119132541}

Of all of the formerly efficient portfolios consisting of four stocks, only one of them still remains efficient when we add the risk-free asset. This portfoliocorresponds to the **point of tangency** of the new straight-line efficient frontier to the former efficient frontier. This portfolio consists only of the 4 risky assets.

How can we find weights of this special portfolio? It offers the best trade-off between risk and return among all other portfolios composed only of these four risky stocks. To find it, we need to maximize the slope of the new efficient frontier. I.e. we need to maximize the **Sharpe ratio**:

$$\theta=\frac{\mu_p-r_f}{\sigma_p}$$

Recall that the Sharpe ratio is the expected return on portfolio in excess to the risk-free rate per unit of portfolio risk.

When only risky assets exist in the economy, the whole expected return is the reward for risk taking (you can only earn some money if you undertake the risk). However, if the risk-free asset exist in the economy you can earn some return without taking any risk (this return is our $r_f$).

Thus, the difference between the expected return on your portfolio and the return on risk-free asset is your reward for taking risk (part of your return which you have earned due to undertaking some risk).

By dividing with portfolio risk $\sigma_p$, we obtain excess return expected to be received for each unit of risk taken.

Tangency portfolio is obtained by **maximizing the Sharpe ratio**. Thus the name Maximum Sharpe Ratio of MSR portfolio.

Recall that whenever we use function <font color='DodgerBlue'><b>sco.minimize</b></font> for maximization, we need to put the minus sign in front of the objective function. Recall also that each portfolio needs to satisfy the budget constraint. Since the solver is meant to minize, we minize the negative of the Sharpe ratio in order to actually maximize it.

In [ ]:
result1 = sco.minimize(lambda w: -(qt.per(w,retM)-qt.expRet(retMrf.rf))/qt.pV(w,retM,vol=True),[0.25]*4,
                       constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
result1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.7232631991293657
       x: [ 7.085e-01  8.616e-01 -6.240e-01  5.390e-02]
     nit: 13
     jac: [-3.340e-03 -3.334e-03 -3.346e-03 -3.326e-03]
    nfev: 66
    njev: 13

Thus the optimal Sharpe ratio is (keep in mind that we have put minus sign in front of the objective function):

In [ ]:
-result1.fun

1.7232631991293657

while the expected return and volatility of this portfolio are, respectively:

In [ ]:
print("Expected Return :",qt.per(result1.x,retM)*100)
print("volatility: ",qt.pV(result1.x,retM,vol=True)*100)

Expected Return : 52.80814498739843
volatility:  30.585155015396325


Let us plot this portfolio on the efficient frontier. If our calculations are correct, it should appear at the point of tangency for the two efficient frontiers (with and without the risk-free asset):

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=sigmaRF,y=miRF,line=dict(color='Blue',width=3),name='4 stocks and rf'))
fig.add_trace(go.Scatter(x=sigma4,y=mi4,line=dict(color='Crimson',width=3),name='4 stocks'))
fig.add_trace(go.Scatter(x=[qt.pV(result1.x,retM,vol=True)],y=[qt.per(result1.x,retM)],
                         marker=dict(color='Gold',line=dict(color='Black',width=2),size=10),showlegend=False))
fig.update_layout(xaxis=dict(title_text='\$\sigma_p$',range=[-0.01,0.4],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu_p$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

Before we move on, let us quickly define the function which calculates the optimal weights for the portfolio with maximal Sharpe's ratio:

Let us try it out on our previous example:

In [ ]:
qt.maxSharpe(retM,retMrf.rf)

{'w': array([ 0.70853206,  0.86148784, -0.62392028,  0.05390038]),
 'er': 0.5280529081816135,
 'vol': 0.3058349879404123,
 'sharpe': 1.723263197039146}

The function allows us to take the assumption about the risk-free rate or to use the data. For example, what would be the optimal portfolio which maximizes Sharpe's ratio if risk-free rate was 2% annually? Here is the answer:

In [ ]:
qt.maxSharpe(retM,0.02)

{'w': array([ 0.82858211,  1.        , -0.85162485,  0.02304275]),
 'er': 0.6178760674609554,
 'vol': 0.35892143218995737,
 'sharpe': 1.6657574996650866}

### <font color='MediumVioletRed' style="font-size:20px"><b>Two-fund separation theorem</b></font>

When we add a risk-free rate into our portfolio, new efficient frontier is a de-facto a straight line starting from the point of the risk-free investment and passing through the tangency portfolio. Any portfolio on the straght line is a combination of the risk-free asset and tangency portfolio!

Points above the tangency portfolio have the expected return higher than the tangency portfolio. They are obtained when we sell short treasuries and purchase tangency portfolio stocks in the proportion entering into the tangency portfolio.

Portfolios on the straight line consist of two "funds": a riskless "fund" that has return $r_f$ and a risky "fund" with return $r_T$. Suppose, further, that the fraction of money invested into the risky fund (i.e. into the tangency portfolio) is $\alpha$ so that the fraction $1-\alpha$ is invested into riskless fund. In that case:

1. The expected return on such portfolio is:

\begin{align*}
\mu_p & = \mathbb{E}(r_p)=(1-\alpha)r_f+\alpha \mathbb{E}(r_T) \\
& \equiv (1-\alpha)r_f+\alpha \mu_T = r_f + \alpha \, (\mu_T - r_f)
\end{align*}

2. Variance of such portfolio is (variance and covariance tems involving risk-less asset are zero):

$$\sigma_p^2 =\mathbb{V}(r_p)=\alpha^2\mathbb{V}(r_T)+(1-\alpha)^2\mathbb{V}(r_f)+2\,\alpha(1-\alpha)\,\mathbb{Cov}(r_f,\,r_T)=\alpha^2\, \sigma_T^2$$

from which follows that the volatility of the two-fund portfolio is:

$$\sigma_p=\alpha\, \sigma_T$$

Combining the expression for volatility and the expected return, we find that (we denoted by $\theta$ maximum Sharpe ratio, i.e. ratio corresponding to the tangency portfolio):

$$\mu_p =r_f + \sigma_p \, \frac{\mu_T -r_f}{\sigma_T} = r_f + \sigma_p \, \theta$$

This equation, simply, says that all efficient portfolios in the presence of a risk-free asset lie on the straight line with the slope $\theta$, i.e. they all have the same, maximal Sharpe ratio.

Sharpe ratio, $\theta$, measures the excess return provided by a portfolio of risky assets (stocks) with respect to the risk free investment, per unit of risk taken. The higher the Sharpe ratio, the better is our investment in risky asset. This is because we can, for the same amount of risk, earn higher expected return. The task of a portfolio manager is, therefore, to maximize the Sharpe ratio of the portfolio.

This equation has an important economic meaning as well. An investor is expecting to receive a risk premium $\sigma_p\,\theta$ in order to take some risk. The more risk she is willing to take the larger is the compensation that she requests.

What is economic implication of this conclusions? - If there is a riskless asset in an economy, a rational investor who trades off risk versus the expected return would optimally invest only in the risky portfolio that provides the highest Sharpe ratio and in riskless asset. His/hers position can be anywhere along the new (straight line) efficient frontier, depending on his/hers risk appetite (preferences).

This conclusion is known as **Two-fund separation theorem**.

## <font color='Orange' style="font-size:25px"><b>Ex-post performance of different portfolios</b></font>

Up to now, we have defined different portfolios using the modeling part of the dataset. Now, we test their ex-post performances on the testing part of the dataset.

### <font color='MediumVioletRed' style="font-size:20px"><b>Static trading strategy</b></font>

In the static trading strategy we start by assuming a certain initial portfolio weights. Based on these weights at the beginning of the trading period (i.e. at the beginning of the testing sample) we purchase of sell short the corresponding assets.

We keep the number of shares thus obtained constant until the end of the investment period (i.e. until the end of the testing sample). This is why this strategy is commonly called **buy-and-hold**:

In table below, we present steps in static trading strategy:

|Step|Description|
|---|---|
|1|Choose the initial amount which you want to invest $V$|
|2|Perform portfolio optimization to find the optimal portfolio weights, $\alpha_j$|
|3|Determine the amount of money which you want to invest in each stock in your portfolio, $v_j$|
|4|Determine the number of shares for each stock that you need to buy or sell short, $n_j$|
|5|Display graphically value of your investement for each time step, $V_t$|

#### <font color='MediumPurple' style="font-size:16px"><b>An example of a static strategy</b></font>

Let us use static trading strategy to compare ex-post performance of MVP with the performance of the efficient portfolio with target return of 15% annually, assuming that we have invested 100 000 dollars into both of them. By making this assumption, we have completed the first step - initial investment is:

In [ ]:
V=100000

Next, we need to determine weights for both of these two portfolios. For that we are going to need portfolio optimization. The optimization is performed on the modeling sample. Returns from the modeling sample are stored as variable *retM*. Vectors of optimal weights for the two strategies are, respectively

In [ ]:
wMVP = qt.mvp(retM)['w']
w15=qt.targetP(retM,0.15)['w']

In [ ]:
wMVP# Optimal weights for the MVP portfolio

array([0.14754181, 0.21792851, 0.44006474, 0.19446494])

In [ ]:
w15 #Optimal weights for portfolio with the target expected return of 15 percent per annum

array([0.2047414 , 0.27680281, 0.3344906 , 0.1839652 ])

In the step three we need to determine the amount of money invested in each stock, $v_j$. Since weights are defined as:

$$\alpha_j=\frac{v_j}{V}$$

if we multiply optimal weights with the initial investment $V$ we obtain $v_j$:

In [ ]:
vMVP = wMVP*V #Vector of money invested into each asset
v15 = w15*V

In [ ]:
vMVP

array([14754.18055975, 21792.85064158, 44006.47447018, 19446.49432848])

In [ ]:
v15

array([20474.13992382, 27680.28051963, 33449.05976375, 18396.51979279])

Next we need to determine the number shares for each constituent that corresponds to the amount of money invested into that asset. To find these quantities that we denote as $n_j$, we first need to find the initial prices in the testing sample for each stock. They correspond to the prices that we need to pay (or or by sell-selling) when we initially enter into the portfolio. The vector of initial prices is $p_0$:

In [ ]:
p0=testingP.iloc[0,:4] #Vector of initial stoc prices
p0

AMZN    371.510010
MSFT     37.889999
IBM     183.899994
AAPL     75.807098
Name: 2014-3-13, dtype: float64

The number of shares are obtained, simply, by dividing the amount invested into a particular assets by the corresponding initial share price:

In [ ]:
nMVP =vMVP/p0
n15=v15/p0

Thus, number of stocks for MVP is:

In [ ]:
nMVP

AMZN     39.714086
MSFT    575.161019
IBM     239.295682
AAPL    256.526034
Name: 2014-3-13, dtype: float64

While number of stocks for portfolio with target expected return of 15% annually:

In [ ]:
n15

AMZN     55.110601
MSFT    730.543177
IBM     181.887226
AAPL    242.675425
Name: 2014-3-13, dtype: float64

Please note that here we assume that we can buy a fractional number of stocks (this is feasible now!). There are also optimization methods which consider integer number of stocks or even round lots (integer multiples of 100s of stocks), but we won't discuss them here (you can check the Chapter 4 of the **Robust Portfolio Optimization** book).

Now, since we have the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two:

In [ ]:
VMVP=testingP.iloc[:,:4]@nMVP
V15 = testingP.iloc[:,:4]@n15

Finally, let us compare them graphically:

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=testingP.index,y=VMVP,line=dict(color='Blue',width=3),name='MVP'))
fig.add_trace(go.Scatter(x=testingP.index,y=V15,line=dict(color='Crimson',width=3),name='Target 15%'))
fig.update_layout(xaxis=dict(title_text='Date',zerolinecolor='Black'),yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
                  title=dict(text="Comparing 2 static strategies",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

As you can see portfolio with the target expected return of 15% outperforms MVP not by very much. Let us compare their values at the end of trading sample numerically:

In [ ]:
V15[-1]-VMVP[-1]

1256.3667427462642

This is roughly 1.3 percent of the initial investment. Note that in this exercise we are not taking into the account transaction costs.

#### <font color='MediumPurple' style="font-size:16px"><b>In a static strategy weights change over time</b></font>

Since we don't perform any trading (i.e. since we keep the number of shares invested constant) people might assume that weights are unchanged as well. But, value of your investment is determined by two factors - price and quantity of stocks.

Thus, although the quantity of shares invested is unchanged, value of our investment in each stock is typically changing over time due to change in prices. Consequently, weights (ratio of the amount invested in stock $j$ and total value of your investment) can change as well!

Let us find weights at the end of trading period for MVP, for example, and compare them with initial weights. As we have explained, weights are ratio of amount invested in stock $j$ (i.e. $v_{jt}=n_j\,p_t$) and the total value of your investment at the same date ($V_t$):

In [ ]:
testingP.iloc[-1,:4]

AMZN    310.350006
MSFT     46.450001
IBM     160.440002
AAPL    110.379997
Name: 2014-12-31, dtype: float64

In [ ]:
nMVP*testingP.iloc[-1,:4]/VMVP[-1]

AMZN    0.116552
MSFT    0.252637
IBM     0.363053
AAPL    0.267759
dtype: float64

On the other hand, the initial weights were:

In [ ]:
wMVP

array([0.14754181, 0.21792851, 0.44006474, 0.19446494])

Thus, as you can see, weights have clearly changed over time.


### <font color='MediumVioletRed' style="font-size:20px"><b>One type of a dynamic trading strategy - keeping weights constant over time</b></font>

If we do not trade, our portfolio may quickly have allocation weights very different from what we have initially considered to be optimal. If we want to keep them constant over time we need to trade. Thus, we need a dynamic strategy. As a result, we need to adjust position $n_j$ over time in order to keep the portfolio weights constant!

Limitation of this strategy is necessity to trade every day. Every transaction bears transaction cost which lower total value of our profit. However, we will disregard them in our examples, but you should be aware of the fact that they do exist in reality.

Here is the list of steps that we need to perform in this strategy:

|Step|Description|
|---|---|
|1|Select the initial amount which you want to invest $V$|
|2|Perform portfolio optimization to find the optimal portfolio weights $\alpha_j$|
|3|Transform the list of prices into the list of returns for each stock, i.e. map from $p_j\to r_j$|
|4|Calculate portfolio returns from stocks' returns $r_j\to r_p$|
|5|Calculate value of your portfolio on each date from list of returns - $r_p\to V_p$|

Please note that first two steps are the same as in case of static strategy.

#### <font color='MediumPurple' style="font-size:16px"><b>Example of dynamic strategies that preserve constant weights</b></font>

Let us compare the same two portfolios as in previous example under the same assumption as before - initial investment is 100000.

In [ ]:
V

100000

Also, optimal weights are the same as before:

In [ ]:
wMVP

array([0.14754181, 0.21792851, 0.44006474, 0.19446494])

In [ ]:
w15

array([0.2047414 , 0.27680281, 0.3344906 , 0.1839652 ])

In the next step we need to transform the list of prices from the testing sample into the list of corresponding returns:

In [ ]:
retT=testingP.iloc[:,:4].pct_change() # Returns on individual assets for the testing sample

In [ ]:
retT

,AMZN,MSFT,IBM,AAPL
DATE,,,,
2014-3-13,NaN,NaN,NaN,NaN
2014-3-14,0.006002,-0.005014,-0.009190,-0.011231
2014-3-17,0.003478,0.009284,0.019757,0.003908
2014-3-18,0.009946,0.039422,0.005382,0.008847
2014-3-19,-0.014626,-0.007080,-0.011241,-0.000263
...,...,...,...,...
2014-12-24,-0.010627,-0.006398,-0.002589,-0.004709
2014-12-26,0.019998,-0.005401,0.003213,0.017677
2014-12-29,0.009544,-0.008981,-0.011273,-0.000702


Next, we need to transform list of stocks’ return into the list of portfolio's returns for all returns in the testing sample. This is something that we have already seen at the beginning of our discussion of portfolio optimization - we need to dot.multiply the matrix of individual asset returns with appropriate weights:

In [ ]:
retMVP=(retT@wMVP) # Ex-post reteurns on MVP portfolio if we keep the weights fixed
ret15P=(retT @ w15) # Ex-post reteurns on w15 portfolio if we keep the weights fixed

Before we continue, replace the missing values with 0.

In [ ]:
retMVP[0]=0
ret15P[0]=0

Finally, we need to calculate the value of our portfolio for each date in the testing sample. Since we have a series of returns, we can transform them into a series of cumulative returns by compounding them (i.e. by using cumulative product - method <font color='DeepPink'><b>cumprod</b></font>).

Once when we have cumulative returns, we can find the value of our portfolio at any time t simply by multiplying them with the initial investment of $V$:

In [ ]:
VMVP=(1+retMVP).cumprod()*V
V15=(1+ret15P).cumprod()*V

In order to compare them, as before, we need to present both series graphically:

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=testingP.index,y=VMVP,line=dict(color='Blue',width=3),name='MVP'))
fig.add_trace(go.Scatter(x=testingP.index,y=V15,line=dict(color='Crimson',width=3),name='Target 15%'))
fig.update_layout(xaxis=dict(title_text='Date',zerolinecolor='Black'),yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
                  title=dict(text="Comparison of two dynamic strategies with constant weights",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
fig.show()

Here we get almost the same graph as before. Once again portfolio with target expected return of 15% annually outperforms MVP by:

In [ ]:
V15[-1]-VMVP[-1]

1394.4208143895958

Which is a little bit higher than in the case of static trading strategies with identical initial weights. Having said that, we are ignoring here transaction costs which can be substantial when we trade every day.

There are algorithms which tell us under what circumstances to actually trade when we want to keep a constant fraction of wealth invested over time in the presence of transaction costs.

With this we conclude our little tour of classical Markowitz portfolio optimization. In following classes we will discuss limitations and potential upgrades of this technique.